In [24]:
import vanna
from vanna.remote import VannaDefault
from vanna.openai import OpenAI_Chat
from vanna.chromadb import ChromaDB_VectorStore
import dotenv
import os
import sys
dotenv.load_dotenv()
from nextgen.vanna.client import LlamaLLM
from vanna.chromadb import ChromaDB_VectorStore
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [26]:
config={
    'APIM_SUBSCRIPTION_KEY': os.getenv('APIM_SUBSCRIPTION_KEY'),
    'path': '../database/chroma'
    }

class MyVanna(ChromaDB_VectorStore, LlamaLLM):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        LlamaLLM.__init__(self, config=config)

vn = MyVanna(config=config)

In [27]:
vn.connect_to_sqlite('../database/nextgen.db')

In [6]:
df_ddl = vn.run_sql("SELECT type, sql FROM sqlite_master WHERE sql is not null")

for ddl in df_ddl['sql'].to_list():
  vn.train(ddl=ddl)

Adding ddl: CREATE TABLE proteins (
    "entry" TEXT PRIMARY KEY,
    "entry_name" TEXT,
    "gene_names" TEXT,
    "gene_names_primary" TEXT,
    "gene_names_synonym" TEXT,
    "gene_names_ordered_locus" TEXT,
    "gene_names_orf" TEXT,
    "organism" TEXT,
    "organism_id" TEXT,
    "protein_names" TEXT,
    "proteomes" TEXT,
    "taxonomic_lineage" TEXT,
    "taxonomic_lineage_ids" TEXT,
    "virus_hosts" TEXT,
    "alternative_products_isoforms" TEXT,
    "alternative_sequence" TEXT,
    "erroneous_gene_model_prediction" TEXT,
    "fragment" TEXT,
    "gene_encoded_by" TEXT,
    "length" TEXT,
    "mass" TEXT,
    "mass_spectrometry" TEXT,
    "natural_variant" TEXT,
    "non_adjacent_residues" TEXT,
    "non_standard_residue" TEXT,
    "non_terminal_residue" TEXT,
    "polymorphism" TEXT,
    "rna_editing" TEXT,
    "sequence" TEXT,
    "sequence_caution" TEXT,
    "sequence_conflict" TEXT,
    "sequence_uncertainty" TEXT,
    "sequence_version" TEXT,
    "absorption" TEXT,
    "

Add of existing embedding ID: a61b820d-0a64-5972-b31c-fa4290ab9c7c-ddl
Insert of existing embedding ID: a61b820d-0a64-5972-b31c-fa4290ab9c7c-ddl
Add of existing embedding ID: 9cd894ff-e4b0-5ff9-b5a5-96a425857357-ddl
Insert of existing embedding ID: 9cd894ff-e4b0-5ff9-b5a5-96a425857357-ddl
Add of existing embedding ID: 5f37cdce-88dd-597a-a13b-49225c2f8529-ddl
Insert of existing embedding ID: 5f37cdce-88dd-597a-a13b-49225c2f8529-ddl


Adding ddl: CREATE TABLE sample (
    "run" TEXT PRIMARY KEY,
    "ipas" TEXT,
    "id" TEXT,
    "plate" TEXT,
    "assay" TEXT,
    "evotip" TEXT,
    "well" TEXT,
    "mced" TEXT,
    "consider" TEXT,
    "is_case" BOOLEAN,
    "sex_1_male_0_female" INTEGER,
    "cancer_type" TEXT,
    "subtype" TEXT,
    "stage" TEXT,
    "age" INTEGER,
    "smoking_status" TEXT,
    "notes" TEXT,
    "check" BOOLEAN,
    "group" TEXT
)
Adding ddl: CREATE TABLE measurement (
    "protein_group" TEXT,
    "protein_ids" TEXT,
    "genes" TEXT,
    "citrullination_r" BOOLEAN,
    "run" TEXT,
    "intensity" REAL
)


In [7]:
# The following are methods for adding training data. Make sure you modify the examples to match your database.

# Sometimes you may want to add documentation about your business terminology or definitions.
vn.train(documentation='''
         In the database, the cancer_type column is used to store the type of cancer. The values and their meanings are as follows:
         Breast: Breast cancer
         Esophageal: Esophagus cancer
         Lung: Lung cancer
         Liver: Liver cancer
         PDAC: Pancreatic ductal adenocarcinoma
         Ovarian: Ovarian cancer
         Gastric: Gastric cancer
         Rectal: Rectal cancer
         GEJ: Gastroesophageal junction cancer
         Benign from SoCC cohort: Benign from SoCC cohort
         MERIT Control: MERIT cohort a healthy control
         LEAP Control: LEAP cohort a healthy control
         When your see a name on the right side of this lookup table, you should use the value on the left side of the lookup table to replace it in your query.
''')

vn.train(documentation='''
         In the measurement table, the protein_group column is used to store the name of protein for the intensity of that record.
         The genes column is used to store the genes of the proteins for the intensity of that record.
         The citrullination_r column is used to store the citrullination status of the proteins for the intensity of that record.
         The run column indicate which sample the intensity is measured from. In combination with the protein_group column, it can be used to find the intensity of a specific protein from a specific sample.
         ''')


vn.train(question='What proteins are important for distinguishing between breast and gastric cancer', 
         sql='''SELECT T2.protein_group, T2.intensity, T1.cancer_type FROM sample AS T1 INNER JOIN measurement AS T2 ON T1.run = T2.run WHERE T1.cancer_type IN ('Breast', 'Gastric')
SELECT T2.protein_group, T2.intensity, T1.cancer_type FROM sample AS T1 INNER JOIN measurement AS T2 ON T1.run = T2.run WHERE T1.cancer_type IN ('Breast', 'Gastric')''')

# You can also add SQL queries to your training data. This is useful if you have some queries already laying around. You can just copy and paste those from your editor to begin generating new SQL.
# vn.train(sql="SELECT * FROM my-table WHERE name = 'John Doe'")

Adding documentation....


Insert of existing embedding ID: 65d10831-f3f8-5ef4-9d92-97366d3e447a-doc
Add of existing embedding ID: 65d10831-f3f8-5ef4-9d92-97366d3e447a-doc
Insert of existing embedding ID: a207678a-bedc-5ca5-9447-22982d2a3ddf-doc
Add of existing embedding ID: a207678a-bedc-5ca5-9447-22982d2a3ddf-doc


Adding documentation....


'a207678a-bedc-5ca5-9447-22982d2a3ddf-doc'

In [8]:
# At any time you can inspect what training data the package is able to reference
training_data = vn.get_training_data()
training_data

,id,question,content,training_data_type
0,09f90530-e190-5453-86cc-5a778b3f5142-sql,what is expression of protein P02649 in breast...,SELECT T2.intensity FROM sample AS T1 INNER JO...,sql
0,a61b820d-0a64-5972-b31c-fa4290ab9c7c-ddl,None,"CREATE TABLE proteins (\n ""entry"" TEXT PRIM...",ddl
1,9cd894ff-e4b0-5ff9-b5a5-96a425857357-ddl,None,"CREATE TABLE sample (\n ""run"" TEXT PRIMARY ...",ddl
2,5f37cdce-88dd-597a-a13b-49225c2f8529-ddl,None,"CREATE TABLE measurement (\n ""protein_group...",ddl
0,65d10831-f3f8-5ef4-9d92-97366d3e447a-doc,None,"\n In the database, the cancer_type co...",documentation
1,a207678a-bedc-5ca5-9447-22982d2a3ddf-doc,None,"\n In the measurement table, the prote...",documentation


In [29]:
# vn.ask("What are names of proteins with citrullination in breast cancer?")
# vn.ask("what is expression of protein P02649 in breast cancer?", auto_train=False)
sql , df, _ = vn.ask("what protein is related to cancer?", auto_train=False, visualize=False)

Number of requested results 10 is greater than number of elements in index 1, updating n_results = 1
Number of requested results 10 is greater than number of elements in index 3, updating n_results = 3
Number of requested results 10 is greater than number of elements in index 2, updating n_results = 2


SQL Prompt: [{'role': 'system', 'content': 'You are a SQLite expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nCREATE TABLE proteins (\n    "entry" TEXT PRIMARY KEY,\n    "entry_name" TEXT,\n    "gene_names" TEXT,\n    "gene_names_primary" TEXT,\n    "gene_names_synonym" TEXT,\n    "gene_names_ordered_locus" TEXT,\n    "gene_names_orf" TEXT,\n    "organism" TEXT,\n    "organism_id" TEXT,\n    "protein_names" TEXT,\n    "proteomes" TEXT,\n    "taxonomic_lineage" TEXT,\n    "taxonomic_lineage_ids" TEXT,\n    "virus_hosts" TEXT,\n    "alternative_products_isoforms" TEXT,\n    "alternative_sequence" TEXT,\n    "erroneous_gene_model_prediction" TEXT,\n    "fragment" TEXT,\n    "gene_encoded_by" TEXT,\n    "length" TEXT,\n    "mass" TEXT,\n    "mass_spectrometry" TEXT,\n    "natural_variant" TEXT,\n    "non_adjacent_residues" TEXT,\n    "non_stan

In [23]:
# need second agent to do the data analysis
df

,protein_group
0,P02649
1,P02649
2,P02649
3,P02649
4,P02649
...,...
691421,Q9Y573
691422,Q9Y573-2
691423,P57071-2
691424,P57071-5


In [ ]:
prompt = '''
You are data analysis expert. You are give a question and corresponding data. You need to analyze the data and answer the question.

Question: What is expression of protein P02649 in breast cancer?
Data:
'''